<h1> Now let get to "PREPROCESSING" the Data </h1>

In [1]:
from IPython.display import display
import pandas as pd
import numpy as np

In [2]:
#Import Dataset into Dataframe
df=pd.read_csv("redcom_1.csv")
df



,body,score,permalink
0,"Perhaps he would, but it would still be becaus...",1,take her out recording appears to capture trump
1,It’s also not a fair trial. Why can’t the repu...,1,take her out recording appears to capture trump
2,"Oil companies, pharmaceutical companies are wi...",1,a senator wants to unilaterally release
3,okay so i clicked on this expecting to hear th...,1,take her out recording appears to capture trump
4,Some states treat your house differently (Cast...,1,organized breakandenter gangs arrive as tourists
...,...,...,...
13985,&gt; anti-Zionism is inherently antisemitic.\n...,1,israel ambassador to us says radical left college
13986,"Well, a lot of right-wing Israeli Jews (the on...",4,israel ambassador to us says radical left college
13987,I read the comment and didn't say anything abo...,-3,fox friends warns of muslim enclaves in the west
13988,&gt; Israel-Palestine debate giving people a s...,0,israel ambassador to us says radical left college


<h1> Task 1: remove all moderator Bot comments "I am a bot, and this action was performed automatically"</h1>
<h2>And remove duplicated row by checking in 'body' column</h2>

In [3]:
# Syntax to drop rows that contain a set of strings 
df = df[~df.body.str.contains('I am a bot')]

# Removing Duplicates
df.body.duplicated().sum()              #FInd and return no  of duplicates in a column
df.loc[df.body.duplicated()]            #locate and display duplicates

#Removes all rows of data with duplicates found in Body, (inplace) ensure the changes reflect on the DF 
df.drop_duplicates(subset=['body'], inplace=True)   

df.shape


C:\Users\malik\miniconda3\envs\tensor2.5\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


(13816, 3)

In [4]:
# Check manually
#display(df)
#df.to_csv("check_2.csv", index=False)    #Save final Dataset into a csv

# ...

<h1> Task 2: Remove all Numbers, Punctuation and non-alphabet Characters, Additional spaces and Removal of 'Stop-words' </h1>
<h2> Add lemmitization using WordNet Lemmatizer from NLTK</h2>
<h2> Tokenize the words using Word_Tokenize from NLTK</h2>
<h3> Note: We will not be stemming words in this project. Stemming sometime create unknown words(spelling Errors). </h3>
<h3> We need a high degree of precision at every level so its best to use a Lemmatizer </h3>
<h3> S.N: Vader can detect Case sensitive sentiment so its best not tamper with case sensitivity</h3>


In [5]:
import re                                       #regular expressions library
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk import pos_tag, word_tokenize                #Both to be used in conjuction with lemmatizer function
nltk.download('punkt')                              #needed for tokenizer
nltk.download('averaged_perceptron_tagger')         #needed for tokenizer
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\malik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\malik\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\malik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\malik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
""" Building the Stopword List """
stop_words = stopwords.words('english')      #store downloaded stopword list in variable
#print(stop_words)
#print(len(stop_words))

from string import punctuation      #import punctuation list

# Add punctuations to stop word list
for x in punctuation:
    stop_words.append(x)


#Additional stop word list
more = ['tl', 'dr', 'drs', 'gt']
for x in more:
    stop_words.append(x)

# More stopword in text document
"""with open('stopwordlist.txt', 'r') as s:   #stopword list maybe too aggressive :(
    for x in s:
        stop_words.append(x.strip())"""
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

<h1> Add POS TAGGING FUCNTION </h1>
<h2> reexamine data, consider spelling , consider Negators, Note: '. & ,' still present </h2>

In [17]:
## POS Tag function works in conjuction with lemmatizer function, this is a rather difficult but necessary setup
## Note: stemming is much easier to perform but ineffective for this usecase
## See https://www.kaggle.com/alvations/basic-nlp-with-nltk for more details on workings

wnl = WordNetLemmatizer()
def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. (Postag function)"""
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n'

def lemmatize_sent(raw_text): 
    """Lemmatize Function containing (POSTAG funtion) to be utilized by preprocess function """
    # Text input is string, returns lowercased strings.
    # word_tokenize is the function from nltk that tokenized/splits our texts
    # pos_tag function from NLTK that performs parts of speech assignment
    # you can also add word.lower() as an args if necessary
    return [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(word_tokenize(raw_text))]


In [18]:
def preprocess(raw_text):
    """ Preprocessing Steps """
    raw_text = str(raw_text)
    
    #regular expression keeping only letters and addressing some negations 
    raw_text = re.sub("[^A-Za-z]", " ", raw_text)
    raw_text = re.sub(r"what's", "what is ", raw_text)
    raw_text = re.sub(r"aren't", "are not ", raw_text)
    raw_text = re.sub(r"wasn't", "was not ", raw_text)
    raw_text = re.sub(r"haven't", "have not ", raw_text)
    raw_text = re.sub(r"hasn't", "has not ", raw_text)
    raw_text = re.sub(r"hadn't", "had not ", raw_text)
    raw_text = re.sub(r"won't", "will not ", raw_text)
    raw_text = re.sub(r"wouldn't", "would not ", raw_text)
    raw_text = re.sub(r"don't", "do not ", raw_text)
    raw_text = re.sub(r"doesn't", "does not ", raw_text)
    raw_text = re.sub(r"didn't", "did not ", raw_text)
    raw_text = re.sub(r"can't", "can not ", raw_text)
    raw_text = re.sub(r"shouldn't", "should not ", raw_text)
    raw_text = re.sub(r"mightn't", "might not ", raw_text)
    raw_text = re.sub(r"mustn't", "must not ", raw_text)
    raw_text = re.sub(r"\'s", " is", raw_text)
    raw_text = re.sub(r"\'ve", " have ", raw_text)
    raw_text = re.sub(r"can't", "cannot ", raw_text)
    raw_text = re.sub(r"n't", " not ", raw_text)
    raw_text = re.sub(r"i'm", "i am ", raw_text)
    raw_text = re.sub(r"\'re", " are ", raw_text)
    raw_text = re.sub(r"\'d", " would ", raw_text)
    raw_text = re.sub(r"\'ll", " will ", raw_text)

    return " ".join([word for word in lemmatize_sent(raw_text) 
            if word not in stop_words])
            #and not word.isdigit()   

"""
#Alternatetively for a Simpler stemming or lemmatization:: use the function below 
def preprocessor(raw_text):
    #"" Preprocessing Steps ""
    raw_text = str(raw_text)
    #raw_text = raw_text.lower()        #remove case sensitivity if neccesary
    
    #regular expression keeping only letters and reconstrusting some words
    raw_text = re.sub("[^A-Za-z]", " ", raw_text)
    raw_text = re.sub(r"what's", "what is ", raw_text)
    raw_text = re.sub(r"\'s", " is", raw_text)
    raw_text = re.sub(r"\'ve", " have ", raw_text)
    raw_text = re.sub(r"can't", "cannot ", raw_text)
    raw_text = re.sub(r"n't", " not ", raw_text)
    raw_text = re.sub(r"i'm", "i am ", raw_text)
    raw_text = re.sub(r"\'re", " are ", raw_text)
    raw_text = re.sub(r"\'d", " would ", raw_text)
    raw_text = re.sub(r"\'ll", " will ", raw_text)



    # split words -> thus convert string into list ( 'hello world' -> ['hello', 'world'])
    words = raw_text.split()

    cleaned_words = []
    wnl = WordNetLemmatizer()         #plug in here any other stemmer or lemmatiser you want to try out
    
    # remove stopwords
    for word in words:
        if word not in stop_words:
            cleaned_words.append(word)
    
    #Alternate method to stemm or lemmatise words easily
    lemmatized_words = []
    for word in cleaned_words:
        word = wnl.lemmatize(word)     #lemmatize the words if you are using a lemmatizer
        lemmatized_words.append(word)
    
    # converting list back to string
    return " ".join(lemmatized_words)
   
   """


'\n#Alternatetively for a Simpler stemming or lemmatization:: use the function below \ndef preprocessor(raw_text):\n    #"" Preprocessing Steps ""\n    raw_text = str(raw_text)\n    #raw_text = raw_text.lower()        #remove case sensitivity if neccesary\n    \n    #regular expression keeping only letters and reconstrusting some words\n    raw_text = re.sub("[^A-Za-z]", " ", raw_text)\n    raw_text = re.sub(r"what\'s", "what is ", raw_text)\n    raw_text = re.sub(r"\'s", " is", raw_text)\n    raw_text = re.sub(r"\'ve", " have ", raw_text)\n    raw_text = re.sub(r"can\'t", "cannot ", raw_text)\n    raw_text = re.sub(r"n\'t", " not ", raw_text)\n    raw_text = re.sub(r"i\'m", "i am ", raw_text)\n    raw_text = re.sub(r"\'re", " are ", raw_text)\n    raw_text = re.sub(r"\'d", " would ", raw_text)\n    raw_text = re.sub(r"\'ll", " will ", raw_text)\n\n\n\n    # split words -> thus convert string into list ( \'hello world\' -> [\'hello\', \'world\'])\n    words = raw_text.split()\n\n    cl

In [19]:
#testing the function with a sample text#
sample_text = "Hey There! !!'!'!! This is a Sample SOLD'M review, which 2007 123happens {blah}%456 to contain CONTRACT happened punctuations universal rights of right contained."
print(preprocess(sample_text))

hey sample sold review happen blah contain contract happen punctuation universal right right contain


In [20]:
df['body'] = df['body'].apply(preprocess)               # Apply preprocess to body column
df['permalink'] = df['permalink'].apply(preprocess)     # Apply preprocess to permalink column

In [21]:
df.head(20)
df.to_csv("preprocessed_2.csv", index=False)

In [22]:
df.shape

(13816, 3)

# END....